In [1]:
def getcordinate(address): #read file and input cordinate 
    def strtolist(x):
        return x.split()

    f= open(address)
    content = f.read()
    cord=[]
    content_list = content.split("\n")
    content_list = content_list[content_list.index('_atom_site_charge')+1:]
    f.close()

    content_list2= list(filter(None, list(map(strtolist,content_list)))) #remove space and translate str to list

    i=1
    for element in content_list2: #identify every atom by suffixing anumber      
        element[0]=element[0]+'_'+str(i)
        i+=1
    
    return content_list2

In [2]:
def getbondlist(cord_list):  
    import math
    #find all bond:
    bond_list=[]
    def getdistance(eachatom1,eachatom2):
        x=min(abs(float(eachatom1[2])-float(eachatom2[2])),abs(float(eachatom1[2])-float(eachatom2[2])+1),abs(float(eachatom1[2])-float(eachatom2[2])-1))
        y=min(abs(float(eachatom1[3])-float(eachatom2[3])),abs(float(eachatom1[3])-float(eachatom2[3])+1),abs(float(eachatom1[3])-float(eachatom2[3])-1))
        z=min(abs(float(eachatom1[4])-float(eachatom2[4])),abs(float(eachatom1[4])-float(eachatom2[4])+1),abs(float(eachatom1[4])-float(eachatom2[4])-1))
        r=math.sqrt(math.pow(x,2)+math.pow(y,2)+math.pow(z,2))
        return r

    for eachatom1 in cord_list: #bond is considered if the distance between two atom is less than 0.07
        for eachatom2 in cord_list:
            if eachatom1[0].find('O')==0:
                if 0.0001<getdistance(eachatom1,eachatom2)<0.075:
                    bond_list.append([eachatom1[0],eachatom2[0]])
                    
    return bond_list

In [3]:
def getcoefficient(cord_list,bond_list):  
    #find Al list in unit cell 1 1 1 
    Al_list=[] 
    for element in bond_list: #find all Al 
        if element[1].find('Al')==0: # str.find('Al')==0 means 'Al' is at the beginning of a string 
            Al_list.append(element[1]) 
    Al_list=list(set(Al_list))


    #find Al-O-Si list
    Al_O_Si_list=[] 
    for element1 in bond_list: #find Si from bond list that match Al-O-
        for element2 in bond_list:
            if element1[1].find('Al')==0:
                if element1[0]==element2[0] and element1[1]!=element2[1]:
                    Al_O_Si_list.append([element1[1],element1[0],element2[1]])


    #find Al-O-Si-O list
    Al_O_Si_O_list=[]
    for element1 in Al_O_Si_list: #find O from bond list that match Al-O-Si-
        for element2 in bond_list:
            if element1[2]==element2[1] and element1[1]!=element2[0]:
                    Al_O_Si_O_list.append([element1[0],element1[1],element1[2],element2[0]])

    #find Al-O-Si-O-T list
    Al_O_Si_O_T_list=[]
    for element1 in Al_O_Si_O_list: #find T from bond list that match Al-O-Si-O-
        for element2 in bond_list:
            if element1[3]==element2[0] and element1[2]!=element2[1]:
                    Al_O_Si_O_T_list.append([element1[0],element1[1],element1[2],element1[3],element2[1]])

    #find second neighbor list
    second_neighbors=[]
    for element in Al_list:
        second_neighbors.append([element,[]])

    for element1 in second_neighbors: #merge Al-O-Si-O-Ts with same Al, and only display T
        for element2 in Al_O_Si_O_T_list:
            if element1[0]==element2[0]:
               element1[1].append(element2[4])

    for element in second_neighbors: #delete diplicated T in each element, and we get second neighbour list
        element[1]=list(set(element[1]))


    #count Si and Al numbers as second neighbors of each Al
    T_number_list=[]
    for element in second_neighbors:
        Si_temper=0
        Al_temper=0
        for innerelement in element[1]:
            if innerelement.count('Si')>0:Si_temper+=1
            if innerelement.count('Al')>0:Al_temper+=1
        T_number_list.append([element[0],Si_temper,Al_temper])


    #sum up the number of second neighbors through all Al
    Si_number=0
    Al_number=0

    for element in T_number_list:
        Si_number+=element[1]
        Al_number+=element[2]

    print("Second Neighbor Numbers:")   
    print("Si\n",Si_number)   
    print("Al\n",Al_number)
    print("len(Al_list)",len(Al_list))
    coefficient=1-(Si_number/(Si_number+Al_number))/((192-len(Al_list))/192)
    print("Coefficient:\n",coefficient)
    return coefficient

In [4]:
import sys
#sys.stdout = open('C://Cation//NaY64_newrandom//output_random_models_from_nax1.0.txt','wt')

####################read input file####################
import random
import copy
coefficient_list=[]
address='C:\\LTA\\LTA96_my.cif'
cord_list=getcordinate(address)
print('Reading '+address+'\n')
bond_list=getbondlist(cord_list)

cord_list_backup=copy.deepcopy(cord_list)
bond_list_backup=copy.deepcopy(bond_list)
######generate n random models ###########################
m=0
coefficient_list=[]
while m<100: #m=number of model to be generated
    print('\nModel ',m+1,':\n')
    cord_list=copy.deepcopy(cord_list_backup)
    bond_list=copy.deepcopy(bond_list_backup)
    i=0 # trial for i-st Al insertion
    
    while i<16: #number of Al are replaced with Si
        print('\nAl ',i+1,':\n')
        cord_list_new=copy.deepcopy(cord_list)
        bond_list_new=copy.deepcopy(bond_list)
        target_Al_index=random.randint(0, 575)
        while(cord_list[target_Al_index][1]!='Al'): #random select a Al
            target_Al_index=random.randint(0, 575)   
        print('Select ',cord_list[target_Al_index][0])
        cord_list_new[target_Al_index][0]=cord_list_new[target_Al_index][0].replace('Al','Si') #rplace Al with SI in cord_list; get new cord list
        cord_list_new[target_Al_index][1]='Si'

        for element in bond_list: #replace Al with Si in bond list; get new bond_list
            if element[1]==cord_list[target_Al_index][0]:
                bond_list_new[bond_list.index(element)][1]=bond_list_new[bond_list.index(element)][1].replace('Al','Si') 
        cord_list=cord_list_new
        bond_list=bond_list_new
        i+=1

    coefficient=getcoefficient(cord_list,bond_list)
    coefficient_list.append(coefficient)
    print(('\nFinal coefficient: ',coefficient))
    print('\nFinal cordinate list\n')
    for element in cord_list:
        for i in element:
            print(i,end=' ')
        print()
    m+=1

print(coefficient_list)
print("average coefficient", sum(coefficient_list)/len(coefficient_list))



Reading C:\LTA\LTA96_my.cif


Model  1 :


Al  1 :

Select  Al_131

Al  2 :

Select  Al_566

Al  3 :

Select  Al_213

Al  4 :

Select  Al_207

Al  5 :

Select  Al_555

Al  6 :

Select  Al_346

Al  7 :

Select  Al_210

Al  8 :

Select  Al_272

Al  9 :

Select  Al_355

Al  10 :

Select  Al_573

Al  11 :

Select  Al_281

Al  12 :

Select  Al_500

Al  13 :

Select  Al_71

Al  14 :

Select  Al_53

Al  15 :

Select  Al_493

Al  16 :

Select  Al_277
Second Neighbor Numbers:
Si
 118
Al
 602
len(Al_list) 80
Coefficient:
 0.719047619047619
('\nFinal coefficient: ', 0.719047619047619)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.

Al_562 Al 0.9063 0.68775 0.5 1.7906 
Al_563 Al 0.5937 0.81225 0.5 1.7906 
Si_564 Si 0.9063 0.81225 0.5 1.8708 
Si_565 Si 0.5937 0.5 0.81225 1.8708 
Si_566 Si 0.9063 0.5 0.81225 1.7906 
Al_567 Al 0.5937 0.5 0.68775 1.7906 
Si_568 Si 0.9063 0.5 0.68775 1.8708 
Si_569 Si 0.5 0.68775 0.9063 1.8708 
Al_570 Al 0.5 0.68775 0.5937 1.7906 
Al_571 Al 0.5 0.81225 0.9063 1.7906 
Si_572 Si 0.5 0.81225 0.5937 1.8708 
Si_573 Si 0.68775 0.5937 0.5 1.7906 
Si_574 Si 0.68775 0.9063 0.5 1.8708 
Si_575 Si 0.81225 0.5937 0.5 1.8708 
Al_576 Al 0.81225 0.9063 0.5 1.7906 

Model  2 :


Al  1 :

Select  Al_70

Al  2 :

Select  Al_56

Al  3 :

Select  Al_421

Al  4 :

Select  Al_563

Al  5 :

Select  Al_338

Al  6 :

Select  Al_195

Al  7 :

Select  Al_71

Al  8 :

Select  Al_492

Al  9 :

Select  Al_206

Al  10 :

Select  Al_203

Al  11 :

Select  Al_496

Al  12 :

Select  Al_339

Al  13 :

Select  Al_355

Al  14 :

Select  Al_417

Al  15 :

Select  Al_428

Al  16 :

Select  Al_68
Second Neighbor Numbers:
Si
 

Oal_543 O 0.5558 0.9442 0.67665 -1.1427 
Oal_544 O 0.9442 0.5558 0.67665 -1.1427 
Oal_545 O 0.5558 0.67665 0.9442 -1.1427 
Oal_546 O 0.9442 0.67665 0.5558 -1.1427 
Oal_547 O 0.9442 0.82335 0.9442 -1.1427 
Oal_548 O 0.5558 0.82335 0.5558 -1.1427 
Oal_549 O 0.67665 0.5558 0.9442 -1.1427 
Oal_550 O 0.67665 0.9442 0.5558 -1.1427 
Oal_551 O 0.82335 0.5558 0.5558 -1.1427 
Oal_552 O 0.82335 0.9442 0.9442 -1.1427 
Si_553 Si 0.5 0.5937 0.68775 1.8708 
Al_554 Al 0.5 0.9063 0.68775 1.7906 
Al_555 Al 0.5 0.5937 0.81225 1.7906 
Si_556 Si 0.5 0.9063 0.81225 1.8708 
Si_557 Si 0.68775 0.5 0.5937 1.8708 
Al_558 Al 0.68775 0.5 0.9063 1.7906 
Al_559 Al 0.81225 0.5 0.5937 1.7906 
Si_560 Si 0.81225 0.5 0.9063 1.8708 
Si_561 Si 0.5937 0.68775 0.5 1.8708 
Al_562 Al 0.9063 0.68775 0.5 1.7906 
Si_563 Si 0.5937 0.81225 0.5 1.7906 
Si_564 Si 0.9063 0.81225 0.5 1.8708 
Si_565 Si 0.5937 0.5 0.81225 1.8708 
Al_566 Al 0.9063 0.5 0.81225 1.7906 
Al_567 Al 0.5937 0.5 0.68775 1.7906 
Si_568 Si 0.9063 0.5 0.68775 1.8708

Select  Al_576

Al  2 :

Select  Al_139

Al  3 :

Select  Al_213

Al  4 :

Select  Al_416

Al  5 :

Select  Al_195

Al  6 :

Select  Al_500

Al  7 :

Select  Al_351

Al  8 :

Select  Al_144

Al  9 :

Select  Al_276

Al  10 :

Select  Al_342

Al  11 :

Select  Al_53

Al  12 :

Select  Al_481

Al  13 :

Select  Al_350

Al  14 :

Select  Al_135

Al  15 :

Select  Al_485

Al  16 :

Select  Al_567
Second Neighbor Numbers:
Si
 128
Al
 592
len(Al_list) 80
Coefficient:
 0.6952380952380952
('\nFinal coefficient: ', 0.6952380952380952)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.14365 -1.1427 
Oal_14 O 0 0.35635 0.14365 -1.1427

Select  Al_493

Al  8 :

Select  Al_281

Al  9 :

Select  Al_428

Al  10 :

Select  Al_347

Al  11 :

Select  Al_571

Al  12 :

Select  Al_496

Al  13 :

Select  Al_409

Al  14 :

Select  Al_484

Al  15 :

Select  Al_430

Al  16 :

Select  Al_134
Second Neighbor Numbers:
Si
 128
Al
 592
len(Al_list) 80
Coefficient:
 0.6952380952380952
('\nFinal coefficient: ', 0.6952380952380952)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.14365 -1.1427 
Oal_14 O 0 0.35635 0.14365 -1.1427 
Oal_15 O 0 0.14365 0.35635 -1.1427 
Oal_16 O 0 0.35635 0.35635 -1.1427 
Oal_17 O 0.14365 0 0.14365 -1.1427 
Oal_18 O 0.14365 0 0.35635 -1.1427 
Oal

...

Select  Al_571

Al  10 :

Select  Al_488

Al  11 :

Select  Al_122

Al  12 :

Select  Al_284

Al  13 :

Select  Al_573

Al  14 :

Select  Al_135

Al  15 :

Select  Al_198

Al  16 :

Select  Al_134
Second Neighbor Numbers:
Si
 118
Al
 602
len(Al_list) 80
Coefficient:
 0.719047619047619
('\nFinal coefficient: ', 0.719047619047619)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.14365 -1.1427 
Oal_14 O 0 0.35635 0.14365 -1.1427 
Oal_15 O 0 0.14365 0.35635 -1.1427 
Oal_16 O 0 0.35635 0.35635 -1.1427 
Oal_17 O 0.14365 0 0.14365 -1.1427 
Oal_18 O 0.14365 0 0.35635 -1.1427 
Oal_19 O 0.35635 0 0.14365 -1.1427 
Oal_20 O 0.35635 0 

Select  Al_280

Al  16 :

Select  Al_70
Second Neighbor Numbers:
Si
 112
Al
 608
len(Al_list) 80
Coefficient:
 0.7333333333333334
('\nFinal coefficient: ', 0.7333333333333334)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.14365 -1.1427 
Oal_14 O 0 0.35635 0.14365 -1.1427 
Oal_15 O 0 0.14365 0.35635 -1.1427 
Oal_16 O 0 0.35635 0.35635 -1.1427 
Oal_17 O 0.14365 0 0.14365 -1.1427 
Oal_18 O 0.14365 0 0.35635 -1.1427 
Oal_19 O 0.35635 0 0.14365 -1.1427 
Oal_20 O 0.35635 0 0.35635 -1.1427 
Oal_21 O 0.14365 0.14365 0 -1.1427 
Oal_22 O 0.35635 0.14365 0 -1.1427 
Oal_23 O 0.14365 0.35635 0 -1.1427 
Oal_24 O 0.35635 0.35635 0 -1.

Select  Al_277

Al  11 :

Select  Al_64

Al  12 :

Select  Al_558

Al  13 :

Select  Al_207

Al  14 :

Select  Al_566

Al  15 :

Select  Al_276

Al  16 :

Select  Al_127
Second Neighbor Numbers:
Si
 118
Al
 602
len(Al_list) 80
Coefficient:
 0.719047619047619
('\nFinal coefficient: ', 0.719047619047619)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.14365 -1.1427 
Oal_14 O 0 0.35635 0.14365 -1.1427 
Oal_15 O 0 0.14365 0.35635 -1.1427 
Oal_16 O 0 0.35635 0.35635 -1.1427 
Oal_17 O 0.14365 0 0.14365 -1.1427 
Oal_18 O 0.14365 0 0.35635 -1.1427 
Oal_19 O 0.35635 0 0.14365 -1.1427 
Oal_20 O 0.35635 0 0.35635 -1.1427 
Oal_21 O 0

Select  Al_141

Al  7 :

Select  Al_65

Al  8 :

Select  Al_357

Al  9 :

Select  Al_265

Al  10 :

Select  Al_338

Al  11 :

Select  Al_417

Al  12 :

Select  Al_210

Al  13 :

Select  Al_355

Al  14 :

Select  Al_412

Al  15 :

Select  Al_71

Al  16 :

Select  Al_481
Second Neighbor Numbers:
Si
 130
Al
 590
len(Al_list) 80
Coefficient:
 0.6904761904761905
('\nFinal coefficient: ', 0.6904761904761905)

Final cordinate list

Oal_1 O 0 0.1167 0.25 -1.1427 
Oal_2 O 0 0.3833 0.25 -1.1427 
Oal_3 O 0.25 0 0.1167 -1.1427 
Oal_4 O 0.25 0 0.3833 -1.1427 
Oal_5 O 0.1167 0.25 0 -1.1427 
Oal_6 O 0.3833 0.25 0 -1.1427 
Oal_7 O 0.1167 0 0.25 -1.1427 
Oal_8 O 0.3833 0 0.25 -1.1427 
Oal_9 O 0 0.25 0.3833 -1.1427 
Oal_10 O 0 0.25 0.1167 -1.1427 
Oal_11 O 0.25 0.1167 0 -1.1427 
Oal_12 O 0.25 0.3833 0 -1.1427 
Oal_13 O 0 0.14365 0.14365 -1.1427 
Oal_14 O 0 0.35635 0.14365 -1.1427 
Oal_15 O 0 0.14365 0.35635 -1.1427 
Oal_16 O 0 0.35635 0.35635 -1.1427 
Oal_17 O 0.14365 0 0.14365 -1.1427 
Oal_18 O 0.14365

Si_278 Si 0.9063 0.5 0.31225 1.8708 
Si_279 Si 0.5937 0.5 0.18775 1.8708 
Al_280 Al 0.9063 0.5 0.18775 1.7906 
Al_281 Al 0.5 0.68775 0.4063 1.7906 
Si_282 Si 0.5 0.68775 0.0937 1.8708 
Si_283 Si 0.5 0.81225 0.4063 1.8708 
Al_284 Al 0.5 0.81225 0.0937 1.7906 
Si_285 Si 0.68775 0.5937 0 1.8708 
Al_286 Al 0.68775 0.9063 0 1.7906 
Al_287 Al 0.81225 0.5937 0 1.7906 
Si_288 Si 0.81225 0.9063 0 1.8708 
Oal_289 O 0 0.1167 0.75 -1.1427 
Oal_290 O 0 0.3833 0.75 -1.1427 
Oal_291 O 0.25 0 0.6167 -1.1427 
Oal_292 O 0.25 0 0.8833 -1.1427 
Oal_293 O 0.1167 0.25 0.5 -1.1427 
Oal_294 O 0.3833 0.25 0.5 -1.1427 
Oal_295 O 0.1167 0 0.75 -1.1427 
Oal_296 O 0.3833 0 0.75 -1.1427 
Oal_297 O 0 0.25 0.8833 -1.1427 
Oal_298 O 0 0.25 0.6167 -1.1427 
Oal_299 O 0.25 0.1167 0.5 -1.1427 
Oal_300 O 0.25 0.3833 0.5 -1.1427 
Oal_301 O 0 0.14365 0.64365 -1.1427 
Oal_302 O 0 0.35635 0.64365 -1.1427 
Oal_303 O 0 0.14365 0.85635 -1.1427 
Oal_304 O 0 0.35635 0.85635 -1.1427 
Oal_305 O 0.14365 0 0.64365 -1.1427 
Oal_306 O 0.